<a href="https://colab.research.google.com/github/ikukang/DeepLearningWithPythonKeras/blob/master/Q%EC%8B%9C%EA%B0%81%EC%8B%A4%EB%AC%B4P1_ImageProcessing_ipynb_%EA%B0%95%EC%9D%B8%EA%B5%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision

[OpenCV-Python Tutorials](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)

[OpenCV-Python Tutorials(한글)](https://opencv-python.readthedocs.io/en/latest/doc/08.imageProcessing/imageProcessing.html)

## Computer Vision의 방법?



```
>>> print(computer_vision == conv_nets)
False
```
전통적인 Computer Vision과 딥 러닝 기반의 Vision문제 해결 방식은 접근방식부터 큰 차이가 있어 보입니다.

그렇다고 해서 많은 매체에서 이야기되는 것처럼 요술방망이같은 알고리즘이 아니라는 것 또한 딥러닝 실무 과정을 통해 확인하셨을 겁니다.
<br>
<br>

블랙박스 알고리즘을 다루는 것 같은 시각**지능** 실무 수업에서 Computer Vision을 돌아보는 이유는 무엇일까요?
<br>
이는 역설적으로 블랙박스 알고리즘을 다루기 때문입니다.

우리가 배웠던 것처럼 딥 러닝 모델은 과적합되기 정말 쉬운 구조입니다.
XAI가 가열차게 연구되고 있지만, 이 분야가 연구되고 있는 이유 또한 딥 러닝 모델이 무엇을 기준으로 학습하고 있는지 대부분의 경우 알 길이 없기 때문입니다.

따라서 딥 러닝 모델이 우리가 원하는 task를 잘 수행할 수 있도록 input 데이터를 잘 만져주고 아키텍처를 잘 핸들링 해주어야 합니다.

이를 위해 이번 실습에서는 Python 기반의 이미지 변형, 특징추출 방식 몇 가지를 간략하게 실습을 통해 소개 드리려 합니다.

## Numpy 튜토리얼과 안내

이미지는 **2D 행렬**(grayscale) 또는 **3D 텐서**(RGB 채널)로 표현됩니다. [OpenCV](https://opencv.org/)라는 가장 유명한 Computer Vision 라이브러리는 이미지에 대한 강력한 편집 기능을 제공하는데, 파이썬의 OpenCV는 사실 C/C++ 라이브러리 구현의 **Wrapper**이기에 매우 빠르게 동작합니다.

추가적으로 OpenCV에서 다루는 모든 이미지는 [Numpy](http://www.numpy.org/) 배열이며, 이는 행렬이나 텐서와 같은 다차원 객체를 다루기에 굉장히 좋은 방법입니다.

OpenCV처럼 Numpy 라이브러리는 Python 명령을 통해 C 언어를 구동시키므로 매우 빠릅니다.

Computer Vision과 관련된 작업을 할 때 이러한 툴들을 통해 할 수 있는 것과 할 수 없는 것을 파악하는 것은 매우 중요합니다.

In [0]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [0]:
!df -h

In [0]:
!cat /proc/meminfo

In [0]:
!cat /proc/cpuinfo

In [0]:
# np는 numpy의 일반적인 약자
import numpy as np

# 리스트로부터 numpy array를 생성

boring_list = [1, 2, 3]
fun_array = np.array(boring_list) # np.array(a_list)는 리스트를 numpy array로 캐스팅합니다.

# Array의 추가를 위한 다양한 함수들이 존재합니다.
ones = np.ones(3)
result = fun_array + ones + np.zeros(3)

print(result)


In [0]:
# 행렬은 어떻게 변환할 수 있을까요?
boring_mtx = [[1, 2], [3, 4]]
ones = np.ones((2, 2)) # (2, 2) 가 원하는 행렬의 모양을 나타냅니다.
identity = np.eye(2) # 단위 행렬 만들기

# Numpy will cast your lists into arrays if need be
try:
    result = boring_mtx + identity
    print(result)
except:
    print("Can I do this?")


In [0]:
# 랜덤 샘플링도 가능합니다.
rando_mtx = np.random.random((100, 100))

# 빌트인 된 다양한 함수 사용
print(np.linalg.norm(rando_mtx))

In [0]:
# 파라미터로 다양한 조건을 부여할 수 있습니다.
print(np.linalg.norm(rando_mtx, axis=0))
print(np.linalg.norm(rando_mtx, ord=2))


In [0]:
# 생각한 그대로 행렬 연산이 가능하게 합니다.
rando_mtx = np.eye(rando_mtx.shape[0]) * rando_mtx


# Broadcasting이라는 강력한 특징이 있습니다.
rando_mtx += 1 # 스칼라 합 연산이 가능하며, 이는 매우 강력합니다.

mtx1 = np.random.random((8, 1, 7, 1))
mtx2 = np.random.random((8, 2, 1, 9))

# 에러가 없고.
res = mtx1 * mtx2 

# 위와 마찬가지의 연산을 합니다.
res2 = mtx1.dot(mtx2)

print(np.array_equal(res, res2), res.shape, res2.shape)

# Numpy Documentary를 꼭 읽어보시길 추천드립니다.

## OpenCV

In [0]:
# 예제 이미지 다운로드
!wget https://upload.wikimedia.org/wikipedia/ko/2/24/Lenna.png -O 'lena.jpg'

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2

# Load in Grayscale
img = cv2.imread('./lena.jpg', 0)

# The underlying representation is a numpy array!
print(type(img))

plt.imshow(img)
plt.show()

img = cv2.imread('./lena.jpg')

plt.imshow(img)
plt.show()

# This would work normally, but it will crash the colab kernel, so don't.
# cv2.imshow('Image', img)


위 이미지의 색이 이상한 것을 알 수 있는데요,

이는 OpenCV는 RGB가 아니라 BGR을 사용하기 때문입니다.
Colab과 Jupyter Notebook 등에서는 OpenCV의 imshow를 사용할 수 없기에 matplotlib를 사용합니다.

In [0]:
imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(imgrgb)
plt.show()

# Or, use fun numpy functions / indexing!
imgrgb = img[:,:,::-1]
plt.imshow(imgrgb)
plt.show()


### 이미지 다루기

#### 이미지 열기

In [0]:
# 직접 lena.jpg 이미지를 열어보세요.

img = cv2.imread('./lena.jpg')

#### 이미지 표시하기

In [0]:
# 직접 lena.jpg를 BGR에서 RGB로 변환하여 matplotlib를 통해 노트북에 표시해보세요.
imgrgb = img[:,:,::-1]
plt.imshow(imgrgb)
plt.show()

#### 이미지 저장하기

In [0]:
# cv2.imwrite를 사용하여 이미지를 lena_saved.jpg로 저장해보세요.
# 아래 셀을 수행하여 잘 저장되었는지 확인합니다.
cv2.imwrite('lena_saved.jpg', imgrgb)

In [0]:
!ls

### 도형 그리기

#### 라인 그리기

In [0]:
#모두 0으로 되어 있는 빈 Canvas(검정색)
canvas = np.zeros((512, 512, 3), np.uint8)
canvas = cv2.line(canvas, (0, 0), (511, 511), (255, 0, 0), 5)

plt.imshow(canvas)

In [0]:
# 초록색으로 "Z"를 그려보세요.
canvas = np.zeros((512, 512, 3), np.uint8)
canvas = cv2.line(canvas, (0, 0), (511, 0), (0, 255, 0), 10)
canvas = cv2.line(canvas, (511, 0), (0, 511), (0, 255, 0), 10)
canvas = cv2.line(canvas, (0, 511), (511, 511), (0, 255, 0), 10)
plt.imshow(canvas)

#### 원 그리기

In [0]:
# 모두 0으로 되어 있는 빈 Canvas(검정색)
canvas = np.zeros((512, 512, 3), np.uint8)
canvas = cv2.circle(canvas, (447,63), 63, (0,0,255), -1)
canvas = cv2.ellipse(canvas, (256,256), (100,50), 0, 0, 180, 255, -1)

plt.imshow(canvas)

In [0]:
# 곰돌이의 얼굴, 귀, 코를 그려보세요.

#### 직사각형 그리기

In [0]:
#모두 0으로 되어 있는 빈 Canvas(검정색)
canvas = np.zeros((512, 512, 3), np.uint8)
canvas = cv2.rectangle(canvas, (384, 0), (510, 128), (0,255,0), 3)

plt.imshow(canvas)

#### 글 쓰기

In [0]:
#모두 0으로 되어 있는 빈 Canvas(검정색)
canvas = np.zeros((512, 512, 3), np.uint8)
canvas = cv2.putText(canvas, 'OpenCV', (10,500), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255), 2)

plt.imshow(canvas)

In [0]:
# 인터넷에서 아무 인물 사진을 받아,
# 안경을 씌우고,
# 밑에 이름을 적어주세요.

img = cv2.imread('./shin.jpg')
# imgrgb = img[:,:,::-1]

imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
imgrgb = cv2.circle(imgrgb,(447,63), 63, (0,0,255), 3)
imgrgb = cv2.putText(imgrgb, 'OpenCV', (10,500), cv2.FONT_HERSHEY_SIMPLEX, 4, (0,0,0), 2)

plt.imshow(imgrgb)

### 이미지 연산

In [0]:
!wget https://opencv-python.readthedocs.io/en/latest/_images/flower1.jpg -O flower1.jpg
!wget https://opencv-python.readthedocs.io/en/latest/_images/flower2.jpg -O flower2.jpg

In [0]:
img1 = cv2.imread('flower1.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.imread('flower2.jpg')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

#### 이미지 더하기

Saturation연산은 한계값을 정하고 그 값을 벗어나는 경우는 모두 특정 값으로 계산하는 방식 입니다.

이미지에서는 0이하는 모두 0, 255이상은 모두 255로 표현하는 것입니다.

modulo연산은 a와 b는 n으로 나눈 나머지 값이 같다라는 의미입니다.

시계를 예로 들면 2와 14는 12로 나눈 나머지가 2로 동일합니다.

이미지에서는 연산의 결과가 256보다 큰 경우는 256으로 나눈 나머지 값으로 결정을 합니다.

In [0]:
#OpenCV 합
plt.imshow(cv2.add(img1,img2))

In [0]:
#Numpy 합
plt.imshow(img1+img2)

#### 이미지 블랜딩하기

In [0]:
#@title Blending Parameters { run: "auto" }

w = 0 #@param {type:"slider", min:0, max:100, step:2}

###########################################################

dst = cv2.addWeighted(img1, float(100-w) * 0.01, img2, float(w) * 0.01, 0)

plt.imshow(dst)

### 이미지 Smoothing

In [0]:
blur = cv2.blur(imgrgb,(10, 10))
gblur = cv2.GaussianBlur(imgrgb,(5,5),0)

plt.imshow(imgrgb), plt.title('Original')
plt.show()
plt.imshow(blur), plt.title('Blurred')
plt.show()
plt.imshow(gblur),plt.title('Gaussian Blur')
plt.show()

### 이미지 Transformation

#### Resizing

In [0]:
# 두 배로 확장
res = cv2.resize(imgrgb, None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
plt.imshow(res)
plt.show()

# 10분의 1로 축소
res = cv2.resize(imgrgb, None,fx=.1, fy=.1, interpolation = cv2.INTER_CUBIC)
plt.imshow(res)
plt.show()

#### Rotating

In [0]:
#@title Transformation Parameters { run: "auto" }

theta_degrees = 240 #@param {type:"slider", min:0, max:360, step:10}
scale = 1.1 #@param {type:"slider", min:0, max:2, step:0.1}
shift_x = -60 #@param {type:"slider", min:-100, max:100, step:2}
shift_y = -100 #@param {type:"slider", min:-100, max:100, step:2}

rows, cols, _ = imgrgb.shape
irows = rows
icols = cols

# rows += int(shift_x)
# cols += int(shift_y)

# 이미지의 중심점을 기준으로 90도 회전 하면서 0.5배 Scale
M= cv2.getRotationMatrix2D((icols/2+shift_x, irows/2+shift_y), theta_degrees, scale)


res = cv2.warpAffine(imgrgb, M,(icols, irows))

titles = ['Original Image','Transformed']
images = [imgrgb, res]

for i in range(2):
    plt.subplot(1,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()

#### Seamless Cloning

In [0]:
!wget https://www.learnopencv.com/wp-content/uploads/2015/02/iloveyouticket.jpg -O iloveyouticket.jpg
!wget https://www.learnopencv.com/wp-content/uploads/2015/02/wood-texture.jpg -O wood-texture.jpg

In [0]:
# Read images : src image will be cloned into dst
im = cv2.imread("wood-texture.jpg")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
obj= cv2.imread("iloveyouticket.jpg")
obj = cv2.cvtColor(obj, cv2.COLOR_BGR2RGB)
plt.imshow(im)
plt.show()
plt.imshow(obj)
plt.show()

In [0]:
# Create an all white mask
mask = 255 * np.ones(obj.shape, obj.dtype)

# The location of the center of the src in the dst
width, height, channels = im.shape
center = (int(height/2), int(width/2))

# Seamlessly clone src into dst and put the results in output
normal_clone = cv2.seamlessClone(obj, im, mask, center, cv2.NORMAL_CLONE)
mixed_clone = cv2.seamlessClone(obj, im, mask, center, cv2.MIXED_CLONE)

# Write results
plt.imshow(normal_clone)
plt.show()
plt.imshow(mixed_clone)
plt.show()

In [0]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/2/23/Big_Tree_with_Red_Sky_in_the_Winter_Night.jpg/800px-Big_Tree_with_Red_Sky_in_the_Winter_Night.jpg -O sky_origin.jpg
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Japan.airlines.b777-300.ja733j.arp.jpg/320px-Japan.airlines.b777-300.ja733j.arp.jpg -O airplane.jpg

In [0]:
src = cv2.imread('airplane.jpg')
src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
dst = cv2.imread('sky_origin.jpg')
dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)

plt.imshow(src)
plt.show()
plt.imshow(dst)
plt.show()

In [0]:
src_mask = np.zeros(src.shape, src.dtype)

poly = np.array([ [4,80], [30,54], [151,63], [274,37],
[315,90], [282,154], [43,132] ], np.int32)

cv2.fillPoly(src_mask, [poly], (255, 255, 255))

plt.imshow(cv2.cvtColor(src_mask, cv2.COLOR_BGR2RGB))

In [0]:
print(src.shape, dst.shape, src_mask.shape)

In [0]:
# This is where the CENTER of the airplane will be placed
center = (600,100)
# Clone seamlessly.
output = cv2.seamlessClone(src, dst, src_mask, center, cv2.NORMAL_CLONE)
# Show result
plt.imshow(output)

### Object Detection

#### Face Detection

In [0]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

img = cv2.imread('shin.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [0]:
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

plt.imshow(img)

In [0]:
# 위 실습에서 받았던 인물 사진에도 적용해보고, 잘 수행되는지 확인해보세요.